In [ ]:
import pickle

import matplotlib.pyplot as plt
import healpy as hp
import numpy as np

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

In [ ]:
data = pickle.load(open("/home/laclavere/Thesis/qubic/qubic/scripts/MapMaking/src/CMM/parametric_d0_DB_test_planck//maps/file_seed1_None_k0.pkl", "rb"))

In [ ]:
data.keys()

In [ ]:
fwhm_in = data["fwhm_in"]
fwhm_out = data["fwhm_out"]
fwhm_rec = data["fwhm_rec"]

Ccmb = HealpixConvolutionGaussianOperator(fwhm_rec[0])
Cdust = HealpixConvolutionGaussianOperator(fwhm_rec[0])
C = [Ccmb, Cdust]
comp_maps_input = data["components_in"]
# comp_maps_input = np.array([C[i](data["components_in"][i]) for i in range(len(C))])
# comp_maps_input = data["components_in_convolved"] 
comp_maps_rec = data["components_iter"]
comp_maps_res = comp_maps_input - comp_maps_rec
center = data["center"]
seenpix = data["seenpix"]
convergence = data["convergence"]


In [ ]:
comp_maps_res[:, seenpix].std(axis=1)

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(len(convergence)):
    plt.plot(convergence[i], label=f"k = {i}")

plt.xlabel("Iteration")
plt.ylabel("Convergence")
plt.legend()
plt.grid()
plt.yscale("log")
plt.legend()

In [ ]:
print(convergence)

In [ ]:
plt.plot(convergence)
plt.xlabel("Iteration")
plt.ylabel("Convergence")
plt.title("PCG Convergence")
plt.yscale("log")
plt.grid()
plt.show()

In [ ]:
print("FWHM in: ", fwhm_in)
print("mean fwhm in: ", np.mean(fwhm_in))
print("FWHM out: ", fwhm_out)
print("FWHM rec: ", fwhm_rec)

In [ ]:
plt.figure(figsize=(12, 6), dpi=300)
name_comp = ["CMB", "Dust"]
STOKES = ["I", "Q", "U"]

# comp_maps_input[:, ~seenpix, :] = hp.UNSEEN
# comp_maps_rec[:, ~seenpix, :] = hp.UNSEEN
# comp_maps_res[:, ~seenpix, :] = hp.UNSEEN


istk = 0
n = 2
sigma_input = np.std(comp_maps_input[:, seenpix, istk], axis=0)
sigma_rec = np.std(comp_maps_rec[:, seenpix, istk], axis=0)
sigma_res = np.std(comp_maps_rec[:, seenpix, istk] - comp_maps_input[:, seenpix, istk], axis=0)

reso = 15

k = 0

plt.figure(figsize=(20, 10))
for icomp in range(2):
    hp.gnomview((comp_maps_input[icomp, :, istk]), rot=center,

                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} input",  
                        cmap="jet",
                        # min = - n * sigma_input[icomp],
                        # max = n * sigma_input[icomp],
                        sub=(2, 3, k + 1))
    
    hp.gnomview((comp_maps_rec[icomp, :, istk]), rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Output",  
                        cmap="jet",
                        # min = - n * sigma_rec[icomp],
                        # max = n * sigma_rec[icomp],
                        sub=(2, 3, k + 2))
    
    hp.gnomview(comp_maps_res[icomp, :, istk], rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Residual",  
                        cmap="jet",
                        # min = - n * sigma_res[icomp],
                        # max = n * sigma_res[icomp],
                        sub=(2, 3, k + 3))
    k += 3

In [ ]:
plt.figure(figsize=(12, 6), dpi=300)
name_comp = ["CMB", "Dust"]
STOKES = ["I", "Q", "U"]

# comp_maps_input[:, ~seenpix, :] = hp.UNSEEN
# comp_maps_rec[:, ~seenpix, :] = hp.UNSEEN
# comp_maps_res[:, ~seenpix, :] = hp.UNSEEN

k = 0
istk = 1

plt.figure(figsize=(20, 10))
for icomp in range(2):
    hp.gnomview((comp_maps_input[icomp, :, istk]), rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} input",  
                        cmap="jet",
    
                        sub=(2, 3, k + 1))
    
    hp.gnomview((comp_maps_rec[icomp, :, istk]), rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Output",  
                        cmap="jet",
         
                        sub=(2, 3, k + 2))
    
    hp.gnomview(comp_maps_res[icomp, :, istk], rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Residual",
                        cmap="jet",
                   
                        sub=(2, 3, k + 3))
    k += 3

In [ ]:
stop

In [ ]:
icomp = 1

test_in = data["components_in"][icomp].copy()
test_out = data["components_iter"][icomp].copy()

test_in[~seenpix, :] = 0
test_out[~seenpix, :] = 0

In [ ]:
def chi2_fwhm(fwhm):
    if fwhm < 0 or fwhm > 0.1:
        return np.inf
    else:
        C = HealpixConvolutionGaussianOperator(fwhm)
        input_maps = C(test_in)
            
        return np.sum((test_out - input_maps) ** 2) / np.sum(input_maps ** 2)

In [ ]:
list_chi2 = []

fwhm = np.logspace(np.log10(0.0005), np.log10(0.05), num=80)
for i in fwhm:
    list_chi2.append(chi2_fwhm(i))

list_chi2 = np.array(list_chi2)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(fwhm, list_chi2)
plt.vlines(fwhm[np.where(list_chi2 == np.min(list_chi2))[0][0]], 0, 2, color="red", linestyle="--", label=f"Best fit : {fwhm[np.where(list_chi2 == np.min(list_chi2))[0][0]].round(4)}")
plt.vlines(fwhm_rec[icomp], 0, 2, color="green", linestyle="--", label=f"FWHM rec : {fwhm_rec[icomp].round(4)}")
plt.title("Chi2 as a function of fwhm")
plt.xlabel("FWHM")
plt.ylabel("Chi2")
plt.grid()
plt.legend()
plt.show()